In [0]:
#Import Python Packages
#from google.colab import drive
#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import pandas as pd
import numpy as np
from scipy.stats import randint
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report


In [0]:
#load the training dataset

trainfile =r'/gdrive/My Drive/CIS_508/Datasets/Insurance Fraud - TRAIN-3000(1).csv'
train=pd.read_csv(trainfile)
train.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [0]:
#load the test data set

testfile=r'/gdrive/My Drive/CIS_508/Datasets/Insurance Fraud -TEST-12900(1).csv'
test=pd.read_csv(testfile)
test.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [0]:
#copy train and test data

traincopy=train.iloc[:,:-1].copy()
testcopy=test.iloc[:,:-1].copy()

traincopy.head()
testcopy.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision


In [0]:
  #lets find the categorical variables

    categorical_features=traincopy.columns[traincopy.dtypes==object].tolist()
  print(categorical_features)

['MONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'BASEPOLICY']


In [0]:
#combination of train and test data

combination=pd.concat([traincopy,testcopy], 

onehotencoded=pd.get_dummies(combination,columns=categorical_features)

x_train=onehotencoded.xs(0)
x_test=onehotencoded.xs(1)
 
print(x_train.shape)
print(x_test.shape)

(2999, 144)
(12918, 144)


In [0]:
#define the parameters

param_random={"max_depth": randint(1,20),
          "min_samples_split":randint(2,200), 
            "min_samples_leaf":randint(1,100), 
                        "max_features":randint(1,50),
                              "max_leaf_nodes":randint(2,100),
                                    "criterion":['gini','entropy'],
                                          "splitter":['best','random']}
                                              

#select the target(fraudfound)

y_train=train.iloc[:,-1]
y_test=test.iloc[:,-1]



In [0]:
rclass=DecisionTreeClassifier()

random_tree=RandomizedSearchCV(rclass,param_random,n_iter=13,cv=15,scoring="accuracy")

random_tree.fit(x_train,y_train)


print("After hypertunig parameters with GridsearchCV:{}".format(random_tree.best_params_))
print("Highest accuracy score we get is:{}".format(random_tree.best_score_))
print("Best estimator are:{}".format(random_tree.best_estimator_))
print("Area under ROC curve:{}".format(random_tree.best_index_))
print("The number of cross-validation splits:{}".format(random_tree.n_splits_))


After hypertunig parameters with GridsearchCV:{'criterion': 'entropy', 'max_depth': 12, 'max_features': 31, 'max_leaf_nodes': 34, 'min_samples_leaf': 62, 'min_samples_split': 171, 'splitter': 'best'}
Highest accuracy score we get is:0.8816272090696899
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=12,
                       max_features=31, max_leaf_nodes=34,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=62, min_samples_split=171,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Area under ROC curve:4
The number of cross-validation splits:15


In [0]:

prediction=pd.DataFrame(random_tree.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

              precision    recall  f1-score   support

          No       0.89      0.99      0.93      2600
         Yes       0.66      0.18      0.29       399

    accuracy                           0.88      2999
   macro avg       0.78      0.58      0.61      2999
weighted avg       0.86      0.88      0.85      2999



In [0]:
#Fraud Detection with Randomized search and Decision Tress Classifier
prediction_test=pd.DataFrame(random_tree.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))
print(confusion_matrix(y_test,prediction_test))

              precision    recall  f1-score   support

          No       0.97      0.95      0.96     12420
         Yes       0.14      0.19      0.16       498

    accuracy                           0.92     12918
   macro avg       0.55      0.57      0.56     12918
weighted avg       0.94      0.92      0.93     12918

[[11813   607]
 [  402    96]]


In [0]:
#model 2- try the ranges
    #define the parameters for precision and recall

param_random={"max_depth": randint(100,500),
          "min_samples_split":randint(250,600), 
           "splitter":['best','random']
        
                        }

rclass_pr=DecisionTreeClassifier()



random_tree_pr=RandomizedSearchCV(rclass_pr,param_random,n_iter=1)

random_tree_pr.fit(x_train,y_train)


print("After hypertunig parameters with GridsearchCV:{}".format(random_tree_pr.best_params_))
print("Highest accuracy score we get is:{}".format(random_tree_pr.best_score_))
print("Best estimator are:{}".format(random_tree_pr.best_estimator_))
print("Area under ROC curve:{}".format(random_tree_pr.best_index_))
print("The number of cross-validation splits:{}".format(random_tree_pr.n_splits_))

prediction=pd.DataFrame(random_tree_pr.predict(x_train),columns=["FRAUDFOUND"])
#Print Classification Report
print(classification_report(y_train,prediction))

#Fraud Detection with Randomized search and Decision Tress Classifier
prediction_test=pd.DataFrame(random_tree_pr.predict(x_test),columns=['FRAUDFOUND'])
print(classification_report(y_test,prediction_test))
print(confusion_matrix(y_test,prediction_test))
                                          

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


After hypertunig parameters with GridsearchCV:{'max_depth': 376, 'min_samples_split': 544, 'splitter': 'random'}
Highest accuracy score we get is:0.8329443147715905
Best estimator are:DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=376,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=544,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='random')
Area under ROC curve:0
The number of cross-validation splits:3
              precision    recall  f1-score   support

          No       0.91      0.94      0.93      2600
         Yes       0.52      0.41      0.46       399

    accuracy                           0.87      2999
   macro avg       0.72      0.67      0.69      2999
weighted avg       0.86      0.87      0.86      2999

              prec